# <font color="gold"><b>RNN</b></font>
### <font color="green"><b>Steps performed:</b></font>
<div style="color:cyan;font-weight:bold;">
1) Load dataset<br>
2) visualize the feature column<br>
3) Plot seasonal decompose<br>
4) Split data into Train-Test<br>
5) Scale the data<br>
6) Feeding batches of data to RNN<br>
7) Building LSTM Model <br>
8) Visualizing the fitted model<br>
9) Evaluation batch on test data<br>
10) Forecast using RNN Model<br>
11) Inverse Transformations and Comparison<br>
12) Saving and loading the model<br>
</div>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

In [ ]:
df = pd.read_csv("../Data/Alcohol_Sales.csv", index_col="DATE", parse_dates=True)
df.index.freq="MS"
df.columns = ['Sales']
df.head()

In [ ]:
df.plot(figsize=(18,9));

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decompositions = seasonal_decompose(df['Sales'])
decompositions.plot();

In [ ]:
decompositions.resid.plot(figsize=(18,9));

# <font color="gold"><ins><b>Step4-5: Train-Test Split and Scale Data</b></ins></font>

In [ ]:
train, test = df.iloc[:313], df.iloc[313:]

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

scaler = MinMaxScaler()


# Fit only to train dataset, as Test Dataset's scale is unknown
scaler.fit(train)

scaled_train_data = scaler.transform(train)
scaled_test_data = scaler.transform(test)

In [ ]:
print(f"Train-data size: {len(scaled_train_data)}, Test-data size: {len(scaled_test_data)}")
print(f"Sum: {len(scaled_train_data) + len(scaled_test_data)}")

In [ ]:
scaled_train_data_df = pd.DataFrame(scaled_train_data)
scaled_train_data_df.columns = train.columns
scaled_train_data_df

# <font color="gold"><b>Step6: Feeding batches of data to RNN via <ins>Time Series Generator</ins></b></font>

This class takes in a sequence of data-points gathered at
equal intervals, along with time series parameters such as
stride, length of history, etc., to produce batches for
training/validation.


In [ ]:
# For feeding batches of data to RNN
from keras.preprocessing.sequence import TimeseriesGenerator

# usually equal to the seasonal period cycle, i.e. 7 for day level data, 12 for month level data, etc
n_timesteps = 12
n_features = 1

# since source of data and traget are same, pass scaled train dataset twice
train_generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_timesteps, batch_size=n_features)

In [ ]:
print("Len of train_dataset = len of generator + len of n_timesteps")
print(f" {len(scaled_train_data)} = {len(train_generator)} + {n_timesteps}")

What _TimeseriesGenerator_ does for us is tranform the sequence <br>
> [t1,t2,t3,t4,t5,t6] into <br>
> [t1,t2,t3,t4,t5] -> [t6] <br>
i.e. it takes a sequence of data, transforms it to the above format, i.e. the input format of RNN <br>
and LHS is features, RHS is label for the RNN for each batch <br>

In [ ]:
X, y = train_generator[0]
print("#Length of training data:\n",X.flatten()); print("\n#Prdicting:",y.flatten());

# <font color="gold"><ins><b>Step7: Building the LSTM Model</ins></b></font>
__Building a model__ <br>
Let's first see what we need to do when we want to train a model

- First, we want to decide a model architecture, this is the number of hidden layers and activation functions, etc (___compile___)
- Secondly, we will want to train our model to get all the paramters to the correct value to map our inputs to our outputs (___fit___)
- Lastly, we will want to use this model to do some feed-forward passes to predict novel inputs (___predict___)

### <font color="teal"><b>Input of an LSTM Layer</b></font>

The input to every LSTM layer(_input_shape=(s,t,f)_) must be 3D-Array of data:
- ___Samples___: One sequence is one sample. A batch is comprised of one or more samples
- ___Time-Steps___: One time step is one point of observation in the sample, i.e. at a time, how many point of observations do you need?
- ___Features___: One feature is one observation at a time step

> For example, the model below defines an input layer that expects 1 or more samples, 50 time steps, and 2 features
```python
model = Sequential()
model.add(LSTM(32, input_shape=(50, 2)))
model.add(Dense(1))
```

<a id="Stacked_LSTM"></a>
### <font color="teal"><b>Stacked LSTM</b></font>

Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model
> However, LSTM layer requires a three-dimensional input and LSTMs by default will produce a two-dimensional output as an interpretation from the end of the sequence

`Stacking LSTM hidden layers makes the model deeper, more accurately earning the description as a deep learning technique
It is the depth of neural networks that is generally attributed to the success of the approach on a wide range of challenging prediction problems`

> Additional hidden layers can be added to a Multilayer Perceptron neural network to make it deeper<br>

> The additional hidden layers are understood to recombine the learned representation from prior layers and create new representations at high levels of abstraction<br>

> e.g. from lines to shapes to objects

- _We can address this by setting the <font color="green">return_sequences=True</font> argument on the layer and having the LSTM output a value for each time step in the input data_<br>
- _This allows us to have 3D output from hidden LSTM layer as input to the next_

We can therefore define a Stacked LSTM as follows:
```python
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
```

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM 

model = Sequential()
# Constructing a Vanila LSTM, i.e. LSTM model that has a single hidden layer of LSTM units,
# and an output layer used to make a prediction
model.add((LSTM(150, activation='relu', input_shape=(n_timesteps, n_features))))
model.add(Dense(1))
model.compile(optimizer='adam', loss=customLoss, metrics=['mse', 'mae', 'acc'])#, 'cosine'])
model.summary()

In [ ]:
model.fit_generator(train_generator, epochs=40,verbose=2)
epochs_till_now = [40]

# <font color="gold"><ins><b>Step8: Visualizing the fitted model</ins></b></font>

In [ ]:
model.history.history.keys()

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch);

# <font color="gold"><ins><b>Step9: Evaluation Batch on Test Data</b></ins></font>
Model is working on the monthly data for a sequence of ___(12 hsitory points)___ -> ___(13th point)___ <br>
So we have to have last ___(12 point of train data)___ -> to predict ___(13th step)___

In [ ]:
first_eval_batch = scaled_train_data[-n_timesteps:]

In [ ]:
first_eval_batch = first_eval_batch.reshape((1, n_timesteps, n_features))
first_eval_batch.shape

In [ ]:
model.predict(first_eval_batch)
# This results that, taking n_timesteps values, below given output must be the first value of the test data-set

In [ ]:
scaled_test_data[0]

# <font color="gold"><ins><b>Step10: Predicting using RNN Model</ins></b></font>

In [ ]:
# Holds the predections
test_predictions = []

# last n_timesteps points from the training dataset
first_eval_batch = scaled_train_data[-n_timesteps:]
# Reshaping it to the format RNN expects
current_batch = first_eval_batch.reshape((1, n_timesteps, n_features))

# How far to predict?
for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
scaled_test_data

# <font color="gold"><ins><b>Step11: Inverse Transformations and Comparison</b></ins></font>

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
test.head()

In [ ]:
test['Predictions'] = true_predictions
test.plot(figsize=(12,8));

# <font color="gold"><ins><b>Step12: Saving and loading the model</b></ins></font>

In [ ]:
model.save("./saved_models/LSTM_Alcohol_Sales")

In [ ]:
from keras.models import load_model
new_model = load_model("./saved_models/LSTM_Alcohol_Sales")
new_model.summary()